In [2]:
import os
import json
import argparse
import numpy as np
import config, consts, paths
from decoding.GPT import GPT
from decoding.utils_stim import get_stim
from decoding.utils_resp import get_resp_test
from decoding.StimulusModel import LMFeatures
from encoding.ridge import ridge, bootstrap_ridge
from utils import flatten_list, save_data
import h5py

/home/ckj24/miniforge-pypy3/envs/fmri-algo/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ckj24/miniforge-pypy3/envs/fmri-algo/lib/python3.13/site-packages/torch/__init__.py:1240: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


In [1]:
subject='UTS09'

In [3]:
def convert_to_serializable(downsampled_feat):
    """Convert downsampled feature dictionary to a serializable format."""
    
    serializable_dict = downsampled_feat.tolist()

    return serializable_dict

In [4]:
def get_response(stories, subject):
	"""Get the subject"s fMRI response for stories."""
	#main_path = pathlib.Path(__file__).parent.parent.resolve()
	subject_dir = os.path.join(config.DATA_DIR, "derivative/preprocessed_data/%s" % subject)
	base = subject_dir
	resp = []
	run_on_set = []
	for story in stories:
		resp_path = os.path.join(base, "%s.hf5" % story)
		hf = h5py.File(resp_path, "r")
		resp.extend(hf["data"][:])
		if not run_on_set:
			run_on_set.append(hf["data"][:].shape[0])
		else:
			run_on_set.append(run_on_set[-1]+hf["data"][:].shape[0])
		#print(hf["data"][:].shape[0], "for story:", story)
		hf.close()
	return np.array(resp), run_on_set[:-1]

In [5]:
# load gpt
stories = flatten_list(consts.STORIES)
stories_t =consts.STORIES_test
with open(os.path.join(config.DATA_LM_DIR, "perceived", "vocab.json"), "r") as f:
    gpt_vocab = json.load(f)
gpt = GPT(path = os.path.join(config.DATA_LM_DIR, "perceived", "model"), vocab = gpt_vocab)
features = LMFeatures(model = gpt, layer = config.GPT_LAYER, context_words = config.GPT_WORDS)


In [15]:
# estimate noise model
num_voxels = consts.NUM_VOXELS[subject]
rstim, tr_stats, word_stats = get_stim([stories_t], "story", features)
alphas = np.zeros(num_voxels)
bscorrs = np.zeros([len(config.ALPHAS), num_voxels, config.NBOOTS])
voxels = np.sort(np.argsort(bscorrs)[-config.VOXELS:])
#splits = np.array_split(range(num_voxels), 2)
weights = np.zeros([rstim.shape[1], num_voxels])

In [17]:
rstim.shape

(291, 3072)

In [16]:
zRresp_t = get_resp_test(subject)
zRresp_t.shape

(291, 98336)

In [9]:
zRresp,run_on_set = get_response(stories, subject)
zRresp.shape

(1869, 98336)

In [10]:
run_on_set

[343, 698, 1065, 1465]

In [ ]:
save_location=os.path.join(paths.EM % subject)
#os.makedirs(save_location, exist_ok=True)
#with open(save_location+'/run_on.json', "w") as file:
#    json.dump(run_on_set,file, indent=4)

In [ ]:
#with open(save_location+'/fmri_train.json', "w") as file:
#        json.dump(convert_to_serializable(zRresp),file, indent=4)
#with open(save_location+'/features_train.json', "w") as file:
#        json.dump(convert_to_serializable(rstim),file, indent=4)

In [18]:
save_location=os.path.join(paths.EM % subject)
with open(save_location+'/fmri_test.json', "w") as file:
        json.dump(convert_to_serializable(zRresp_t),file, indent=4)
with open(save_location+'/features_test.json', "w") as file:
        json.dump(convert_to_serializable(rstim),file, indent=4)